Import Objects Scanned Data

In [1]:
import pandas as pd
CSV_Folder_Path = 'C:/Micha/MSTR/MicroStrategyPython/DataExport/'
df_Objects_Scanned = pd.read_csv(CSV_Folder_Path + 'mstr_objects_scanned.csv', sep=';')
df_Objects_Scanned.drop('Unnamed: 0', axis=1, inplace=True)
df_Metrics_to_Metrics_Relations = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Metric') & (df_Objects_Scanned['Dependent Object Type']=='Metric')]
df_Master_Metrics_Relations = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Metric') & (df_Objects_Scanned['Dependent Object Type']!='Metric') &
                                                 (df_Objects_Scanned['Dependent Object Type']!='n.a.')]


Create one single Lookup Reference Dataframe for all Objects

In [2]:
df_Lookup_Source_Objects = df_Objects_Scanned[['Source Object Type', 'Source Object Name', 'Source Object ID']].drop_duplicates()
df_Lookup_Source_Objects = df_Lookup_Source_Objects.rename(columns={'Source Object Type': 'Object Type', 'Source Object Name': 'Object Name', 'Source Object ID': 'Object ID'})

df_Lookup_Dependent_Objects = df_Objects_Scanned[['Dependent Object Type', 'Dependent Object Name', 'Dependent Object ID']].drop_duplicates()
df_Lookup_Dependent_Objects = df_Lookup_Dependent_Objects.rename(columns={'Dependent Object Type': 'Object Type', 'Dependent Object Name': 'Object Name', 'Dependent Object ID': 'Object ID'})

#Unions both tables into one singe table, drop duplicates and update Dataframe Index
df_Lookup_Master = pd.concat([df_Lookup_Source_Objects, df_Lookup_Dependent_Objects]).drop_duplicates()
df_Lookup_Master = df_Lookup_Master.reset_index(drop=True)

Create Lookup Tables of Object Types

In [3]:
df_LU_Table = df_Lookup_Master[(df_Lookup_Master['Object Type']=='Table')].reset_index(drop=True)
df_LU_FACT = df_Lookup_Master[(df_Lookup_Master['Object Type']=='Fact')].reset_index(drop=True)
df_LU_Attribute = df_Lookup_Master[(df_Lookup_Master['Object Type']=='Attribute')].reset_index(drop=True)
df_LU_Metric = df_Lookup_Master[(df_Lookup_Master['Object Type']=='Metric')].reset_index(drop=True)
df_LU_Report = df_Lookup_Master[(df_Lookup_Master['Object Type']=='Report')].reset_index(drop=True)
df_LU_Bulk_Report = df_Lookup_Master[(df_Lookup_Master['Object Type']=='Bulk-Report')].reset_index(drop=True)
df_LU_OLAP_Cube = df_Lookup_Master[(df_Lookup_Master['Object Type']=='OLAP-Cube')].reset_index(drop=True)
df_LU_Document_Dossier = df_Lookup_Master[(df_Lookup_Master['Object Type']=='Document/Dossier')].reset_index(drop=True)
df_LU_Object_Type = df_Lookup_Master[['Object Type']].drop_duplicates().reset_index(drop=True)

Create NxM Relation Tables for Object Relations

In [4]:
df_NxM_Table_Fact = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Table') & (df_Objects_Scanned['Dependent Object Type']=='Fact')]
df_NxM_Table_Attribute = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Table') & (df_Objects_Scanned['Dependent Object Type']=='Attribute')]

df_NxM_Fact_Metric = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Fact') & (df_Objects_Scanned['Dependent Object Type']=='Metric')]

df_NxM_Attribute_Metric = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Attribute') & (df_Objects_Scanned['Dependent Object Type']=='Metric')]
df_NxM_Attribute_Report = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Attribute') & (df_Objects_Scanned['Dependent Object Type']=='Report')]
df_NxM_Attribute_Document_Dossier = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Attribute') & (df_Objects_Scanned['Dependent Object Type']=='Document/Dossier')]
df_NxM_Attribute_OLAP_Cube = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Attribute') & (df_Objects_Scanned['Dependent Object Type']=='OLAP-Cube')]
df_NxM_Attribute_BULK_Report = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Attribute') & (df_Objects_Scanned['Dependent Object Type']=='Bulk-Report')]

df_NxM_Metric_Report = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Metric') & (df_Objects_Scanned['Dependent Object Type']=='Report')]
df_NxM_Metric_Document_Dossier = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Metric') & (df_Objects_Scanned['Dependent Object Type']=='Document/Dossier')]
df_NxM_Metric_OLAP_Cube = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Metric') & (df_Objects_Scanned['Dependent Object Type']=='OLAP-Cube')]
df_NxM_Metric_BULK_Report = df_Objects_Scanned[(df_Objects_Scanned['Source Object Type']=='Metric') & (df_Objects_Scanned['Dependent Object Type']=='Bulk-Report')]

Resolve Metric to Metric Relations

In [5]:
#Recursice Function Start
#########################
def FlattenMetricToMetricsDependencies(Parent_Metric_ID, Saved_Application_Object_Type, Saved_Application_Object_Name, Saved_Application_Object_ID):
    df_Child_Metrics_Relations = df_Metrics_to_Metrics_Relations[(df_Metrics_to_Metrics_Relations['Dependent Object ID'] == Parent_Metric_ID)]
    rows_total_child_metrics_relations = df_Child_Metrics_Relations.shape[0]
    rows_current_child_metric = 0
    global df_NxM_Metric_Report
    global df_NxM_Metric_Document_Dossier
    global df_NxM_Metric_OLAP_Cube
    global df_NxM_Metric_BULK_Report

    while rows_current_child_metric < rows_total_child_metrics_relations:
        df_Child_Metrics_Relations_Row = df_Child_Metrics_Relations.iloc[rows_current_child_metric:rows_current_child_metric + 1]

        Child_Metric_Type = df_Child_Metrics_Relations_Row.iloc[0]['Source Object Type']    
        Child_Metric_Name = df_Child_Metrics_Relations_Row.iloc[0]['Source Object Name']    
        Child_Metric_ID = df_Child_Metrics_Relations_Row.iloc[0]['Source Object ID']    

        dict_New_Metric_Relation_Row = {
                                        "Source Object Type": Child_Metric_Type,
                                        "Source Object Name": Child_Metric_Name,
                                        "Source Object ID": Child_Metric_ID,
                                        "Dependent Object Type": Saved_Application_Object_Type,
                                        "Dependent Object Name": Saved_Application_Object_Name,
                                        "Dependent Object ID": Saved_Application_Object_ID
                                       }

        if Saved_Application_Object_Type == 'Report':
            df_NxM_Metric_Report = pd.concat([df_NxM_Metric_Report, pd.DataFrame([dict_New_Metric_Relation_Row])])
        elif Saved_Application_Object_Type == 'Document/Dossier':
            df_NxM_Metric_Document_Dossier = pd.concat([df_NxM_Metric_Document_Dossier, pd.DataFrame([dict_New_Metric_Relation_Row])])
        elif Saved_Application_Object_Type == 'OLAP-Cube':
            df_NxM_Metric_OLAP_Cube = pd.concat([df_NxM_Metric_OLAP_Cube, pd.DataFrame([dict_New_Metric_Relation_Row])])
        elif Saved_Application_Object_Type == 'Bulk-Report':
            df_NxM_Metric_BULK_Report = pd.concat([df_NxM_Metric_BULK_Report, pd.DataFrame([dict_New_Metric_Relation_Row])])

        rows_current_child_metric = rows_current_child_metric + 1
        FlattenMetricToMetricsDependencies(Child_Metric_ID, Saved_Application_Object_Type, Saved_Application_Object_Name, Saved_Application_Object_ID)
    
#Recursice Function End
#######################
    
rows_total_master_metrics_relations = df_Master_Metrics_Relations.shape[0]
rows_current_master_metric = 0

#Metric Master Rows
while rows_current_master_metric < rows_total_master_metrics_relations:
    df_Master_Metrics_Relations_Row = df_Master_Metrics_Relations.iloc[rows_current_master_metric:rows_current_master_metric + 1]
    
    Parent_Metric_ID = df_Master_Metrics_Relations_Row.iloc[0]['Source Object ID']    
    Saved_Application_Object_Type = df_Master_Metrics_Relations_Row.iloc[0]['Dependent Object Type']    
    Saved_Application_Object_Name = df_Master_Metrics_Relations_Row.iloc[0]['Dependent Object Name']    
    Saved_Application_Object_ID = df_Master_Metrics_Relations_Row.iloc[0]['Dependent Object ID']    
    
    FlattenMetricToMetricsDependencies(Parent_Metric_ID, Saved_Application_Object_Type, Saved_Application_Object_Name, Saved_Application_Object_ID)

    rows_current_master_metric = rows_current_master_metric + 1

Drop Duplicates if there are any and reset Index of Dataframe

In [6]:
df_NxM_Table_Fact = df_NxM_Table_Fact.drop_duplicates().reset_index(drop=True)
df_NxM_Table_Attribute = df_NxM_Table_Attribute.drop_duplicates().reset_index(drop=True)

df_NxM_Fact_Metric = df_NxM_Fact_Metric.drop_duplicates().reset_index(drop=True)

df_NxM_Attribute_Metric = df_NxM_Attribute_Metric.drop_duplicates().reset_index(drop=True)
df_NxM_Attribute_Report = df_NxM_Attribute_Report.drop_duplicates().reset_index(drop=True)
df_NxM_Attribute_Document_Dossier = df_NxM_Attribute_Document_Dossier.drop_duplicates().reset_index(drop=True)
df_NxM_Attribute_OLAP_Cube = df_NxM_Attribute_OLAP_Cube.drop_duplicates().reset_index(drop=True)
df_NxM_Attribute_BULK_Report = df_NxM_Attribute_BULK_Report.drop_duplicates().reset_index(drop=True)

df_NxM_Metric_Report = df_NxM_Metric_Report.drop_duplicates().reset_index(drop=True)
df_NxM_Metric_Document_Dossier = df_NxM_Metric_Document_Dossier.drop_duplicates().reset_index(drop=True)
df_NxM_Metric_OLAP_Cube = df_NxM_Metric_OLAP_Cube.drop_duplicates().reset_index(drop=True)
df_NxM_Metric_BULK_Report = df_NxM_Metric_BULK_Report.drop_duplicates().reset_index(drop=True)

Create Data Files

In [7]:
# Lookup Tables
###############
df_LU_Table = df_LU_Table.rename(columns={'Object Type': 'object_type', 'Object Name': 'table_name', 'Object ID': 'table_id'})
df_LU_Table.to_csv(CSV_Folder_Path + 'lu_table.csv', sep=';', index=False)

df_LU_FACT = df_LU_FACT.rename(columns={'Object Type': 'object_type', 'Object Name': 'fact_name', 'Object ID': 'fact_id'})
df_LU_FACT.to_csv(CSV_Folder_Path + 'lu_fact.csv', sep=';', index=False)

df_LU_Attribute = df_LU_Attribute.rename(columns={'Object Type': 'object_type', 'Object Name': 'attribute_name', 'Object ID': 'attribute_id'})
df_LU_Attribute.to_csv(CSV_Folder_Path + 'lu_attribute.csv', sep=';', index=False)

df_LU_Metric = df_LU_Metric.rename(columns={'Object Type': 'object_type', 'Object Name': 'metric_name', 'Object ID': 'metric_id'})
df_LU_Metric.to_csv(CSV_Folder_Path + 'lu_metric.csv', sep=';', index=False)

df_LU_Report = df_LU_Report.rename(columns={'Object Type': 'object_type', 'Object Name': 'report_name', 'Object ID': 'report_id'})
df_LU_Report.to_csv(CSV_Folder_Path + 'lu_report.csv', sep=';', index=False)

df_LU_Bulk_Report = df_LU_Bulk_Report.rename(columns={'Object Type': 'object_type', 'Object Name': 'bulk_report_name', 'Object ID': 'bulk_report_id'})
df_LU_Bulk_Report.to_csv(CSV_Folder_Path + 'lu_bulk_report.csv', sep=';', index=False)

df_LU_OLAP_Cube = df_LU_OLAP_Cube.rename(columns={'Object Type': 'object_type', 'Object Name': 'olap_cube_name', 'Object ID': 'olap_cube_id'})
df_LU_OLAP_Cube.to_csv(CSV_Folder_Path + 'lu_olap_cube.csv', sep=';', index=False)

df_LU_Document_Dossier = df_LU_Document_Dossier.rename(columns={'Object Type': 'object_type', 'Object Name': 'document_dossier_name', 'Object ID': 'document_dossier_id'})
df_LU_Document_Dossier.to_csv(CSV_Folder_Path + 'lu_document_dossier.csv', sep=';', index=False)

df_LU_Object_Type = df_LU_Object_Type.rename(columns={'Object Type': 'object_type'})
df_LU_Object_Type.to_csv(CSV_Folder_Path + 'lu_object_type.csv', sep=';', index=False)


#Relation Tables
################
df_NxM_Table_Fact = df_NxM_Table_Fact.rename(columns={'Source Object Type': 'source_object_type', 'Source Object Name': 'table_name', 'Source Object ID': 'table_id', 
                                                                                'Dependent Object Type': 'dependent_object_type', 'Dependent Object Name': 'fact_name', 'Dependent Object ID': 'fact_id'})
df_NxM_Table_Fact.to_csv(CSV_Folder_Path + 'nxm_table_fact.csv', sep=';', index=False)


df_NxM_Table_Attribute = df_NxM_Table_Attribute.rename(columns={'Source Object Type': 'source_object_type', 'Source Object Name': 'table_name', 'Source Object ID': 'table_id', 
                                                                                'Dependent Object Type': 'dependent_object_type', 'Dependent Object Name': 'attribute_name', 'Dependent Object ID': 'attribute_id'})
df_NxM_Table_Attribute.to_csv(CSV_Folder_Path + 'nxm_table_attribute.csv', sep=';', index=False)


df_NxM_Fact_Metric = df_NxM_Fact_Metric.rename(columns={'Source Object Type': 'source_object_type', 'Source Object Name': 'fact_name', 'Source Object ID': 'fact_id', 
                                                                                'Dependent Object Type': 'dependent_object_type', 'Dependent Object Name': 'metric_name', 'Dependent Object ID': 'metric_id'})
df_NxM_Fact_Metric.to_csv(CSV_Folder_Path + 'nxm_fact_metric.csv', sep=';', index=False)


df_NxM_Attribute_Metric = df_NxM_Attribute_Metric.rename(columns={'Source Object Type': 'source_object_type', 'Source Object Name': 'attribute_name', 'Source Object ID': 'attribute_id', 
                                                                                'Dependent Object Type': 'dependent_object_type', 'Dependent Object Name': 'metric_name', 'Dependent Object ID': 'metric_id'})
df_NxM_Attribute_Metric.to_csv(CSV_Folder_Path + 'nxm_attribute_metric.csv', sep=';', index=False)


df_NxM_Attribute_Report = df_NxM_Attribute_Report.rename(columns={'Source Object Type': 'source_object_type', 'Source Object Name': 'attribute_name', 'Source Object ID': 'attribute_id', 
                                                                                'Dependent Object Type': 'dependent_object_type', 'Dependent Object Name': 'report_name', 'Dependent Object ID': 'report_id'})
df_NxM_Attribute_Report.to_csv(CSV_Folder_Path + 'nxm_attribute_report.csv', sep=';', index=False)


df_NxM_Attribute_Document_Dossier = df_NxM_Attribute_Document_Dossier.rename(columns={'Source Object Type': 'source_object_type', 'Source Object Name': 'attribute_name', 'Source Object ID': 'attribute_id', 
                                                                                'Dependent Object Type': 'dependent_object_type', 'Dependent Object Name': 'document_dossier_name', 'Dependent Object ID': 'document_dossier_id'})
df_NxM_Attribute_Document_Dossier.to_csv(CSV_Folder_Path + 'nxm_attribute_document_dossier.csv', sep=';', index=False)


df_NxM_Attribute_OLAP_Cube = df_NxM_Attribute_OLAP_Cube.rename(columns={'Source Object Type': 'source_object_type', 'Source Object Name': 'attribute_name', 'Source Object ID': 'attribute_id', 
                                                                                'Dependent Object Type': 'dependent_object_type', 'Dependent Object Name': 'olap_cube_name', 'Dependent Object ID': 'olap_cube_id'})
df_NxM_Attribute_OLAP_Cube.to_csv(CSV_Folder_Path + 'nxm_attribute_olap_cube.csv', sep=';', index=False)


df_NxM_Attribute_BULK_Report = df_NxM_Attribute_BULK_Report.rename(columns={'Source Object Type': 'source_object_type', 'Source Object Name': 'attribute_name', 'Source Object ID': 'attribute_id', 
                                                                                'Dependent Object Type': 'dependent_object_type', 'Dependent Object Name': 'bulk_report_name', 'Dependent Object ID': 'bulk_report_id'})
df_NxM_Attribute_BULK_Report.to_csv(CSV_Folder_Path + 'nxm_attribute_bulk_report.csv', sep=';', index=False)


df_NxM_Metric_Report = df_NxM_Metric_Report.rename(columns={'Source Object Type': 'source_object_type', 'Source Object Name': 'metric_name', 'Source Object ID': 'metric_id', 
                                                                                'Dependent Object Type': 'dependent_object_type', 'Dependent Object Name': 'report_name', 'Dependent Object ID': 'report_id'})
df_NxM_Metric_Report.to_csv(CSV_Folder_Path + 'nxm_metric_report.csv', sep=';', index=False)


df_NxM_Metric_Document_Dossier = df_NxM_Metric_Document_Dossier.rename(columns={'Source Object Type': 'source_object_type', 'Source Object Name': 'metric_name', 'Source Object ID': 'metric_id', 
                                                                                'Dependent Object Type': 'dependent_object_type', 'Dependent Object Name': 'document_dossier_name', 'Dependent Object ID': 'document_dossier_id'})
df_NxM_Metric_Document_Dossier.to_csv(CSV_Folder_Path + 'nxm_metric_document_dossier.csv', sep=';', index=False)


df_NxM_Metric_OLAP_Cube = df_NxM_Metric_OLAP_Cube.rename(columns={'Source Object Type': 'source_object_type', 'Source Object Name': 'metric_name', 'Source Object ID': 'metric_id', 
                                                                                'Dependent Object Type': 'dependent_object_type', 'Dependent Object Name': 'olap_cube_name', 'Dependent Object ID': 'olap_cube_id'})
df_NxM_Metric_OLAP_Cube.to_csv(CSV_Folder_Path + 'nxm_metric_olap_cube.csv', sep=';', index=False)


df_NxM_Metric_BULK_Report = df_NxM_Metric_BULK_Report.rename(columns={'Source Object Type': 'source_object_type', 'Source Object Name': 'metric_name', 'Source Object ID': 'metric_id', 
                                                                                'Dependent Object Type': 'dependent_object_type', 'Dependent Object Name': 'bulk_report_name', 'Dependent Object ID': 'bulk_report_id'})
df_NxM_Metric_BULK_Report.to_csv(CSV_Folder_Path + 'nxm_metric_bulk_report.csv', sep=';', index=False)